In [1]:
#r "nuget:Microsoft.Data.Analysis,0.1.0"
using Microsoft.Data.Analysis;

Installing package Microsoft.Data.Analysis, version 0.1.0.....done!

Successfully added reference to package Microsoft.Data.Analysis, version 0.1.0

In [2]:
PrimitiveDataFrameColumn<DateTime> dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes"); // Default length is 0.
PrimitiveDataFrameColumn<int> ints = new PrimitiveDataFrameColumn<int>("Ints", 3); // Makes a column of length 3. Filled with nulls initially
StringDataFrameColumn strings = new StringDataFrameColumn("Strings", 3); // Makes a column of length 3. Filled with nulls initially

In [3]:
// Append 3 values to dateTimes
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/01"));

In [4]:
DataFrame df = new DataFrame(new List<DataFrameColumn> { dateTimes, ints, strings }); // This will throw if the columns are of different lengths

In [6]:
df

RowCount,Columns
3,"[ [ 2019-01-01 00:00:00Z, 2019-01-01 00:00:00Z, 2019-01-01 00:00:00Z ], [ <null>, <null>, <null> ], [ <null>, <null>, <null> ] ]"


In [7]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.RowCount); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [8]:
df

0,2019-01-01 00:00:00Z,<null>,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-01 00:00:00Z,<null>,<null>


In [9]:
// To change a value directly through df
df[0, 1] = 10; // 0 is the rowIndex, and 1 is the columnIndex. This sets the 0th value in the Ints columns to 10
df

0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-01 00:00:00Z,<null>,<null>


In [10]:
// Modify ints and strings columns by indexing
ints[1] = 100;
strings[1] = "Foo!";
df

0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,100,Foo!
2,2019-01-01 00:00:00Z,<null>,<null>


In [11]:
// Indexing can throw when types don't match.
// ints[1] = "this will throw because I am a string";  
// DataType can be used to figure out the type of data in a column. 
ints.DataType

System.Int32

In [12]:
// Add 5 to ints through the DataFrame
df["Ints"].Add(5, inPlace: true);
df

0,2019-01-01 00:00:00Z,15,<null>
1,2019-01-01 00:00:00Z,105,Foo!
2,2019-01-01 00:00:00Z,<null>,<null>


In [21]:
// We can also use binary operators. Binary operators produce a copy, so assign it back to our Ints column 
df["Ints"] = (ints / 5) * 100;
df

Unhandled exception: (4,3): error CS0650: Bad array declarator: To declare a managed array the rank specifier precedes the variable's identifier. To declare a fixed size buffer field, use the fixed keyword before the field type.
(4,4): error CS0270: Array size cannot be specified in a variable declaration (try initializing with a 'new' expression)

In [14]:
// Fill nulls in our columns, if any. Ints[2], Strings[0] and Strings[1] are null
df["Ints"].FillNulls(-1, inPlace: true);
df["Strings"].FillNulls("Bar", inPlace: true);
df

0,2019-01-01 00:00:00Z,300,Bar
1,2019-01-01 00:00:00Z,2100,Foo!
2,2019-01-01 00:00:00Z,-1,Bar


In [15]:
// To inspect the first row
IList<object> row0 = df[0];
row0

index,
0,2019-01-01 00:00:00Z
1,300
2,Bar


In [16]:
// Filter rows based on equality
DataFrame filtered = df.Filter(strings.ElementwiseNotEquals("Foo!"));
filtered

0,2019-01-01 00:00:00Z,300,Bar
1,2019-01-01 00:00:00Z,-1,Bar


In [17]:
// Sort our dataframe using the Ints column
DataFrame sorted = df.Sort("Ints", ascending: true);
sorted

0,2019-01-01 00:00:00Z,-1,Bar
1,2019-01-01 00:00:00Z,300,Bar
2,2019-01-01 00:00:00Z,2100,Foo!


In [19]:
// GroupBy 
GroupBy groupBy = df.GroupBy("DateTimes");
// Count of values in each group
DataFrame grouped = groupBy.Count(); // Alternatively find the count in just the desired columns
grouped

0,2019-01-01 00:00:00Z,3,3
